# Fine-tune with Pre-trained Models

In practice the dataset we use is relative small, so that we do not train an neural network from scratch, namely staring from random initialized parameters. Instead, it is common to train a neural network on a large-scale dataset and then use it either as an initialization or a fixed feature extractor. On [predict.ipynb](./predict.ipynb) we explained how to do the feature extraction, this tutorial will focus on how to use pre-trained model to fine tune a new network.

The idea of fine-tune is that, we take a pre-trained model, replace the last fully-connected layer with new one, which outputs the desired number of classes and initializes with random values. Then we train as normal except that we may often use a smaller learning rate since we may already very close the final result. 

We will use pre-trained models on the Imagenet dataset to fine-tune the smaller caltech-256 dataset as an example. But note that it can be used to other datasets as well, even for quite different applications such as face identification. 

## Prepare data

We follow the standard protocal to sample 60 images from each class as the training set, and the rest for the validation set. We resize images into 256x256 size and pack them into the rec file. The scripts to prepare the data is as following. 


```sh
wget http://www.vision.caltech.edu/Image_Datasets/Caltech256/256_ObjectCategories.tar
tar -xf 256_ObjectCategories.tar

mkdir -p caltech_256_train_60
for i in 256_ObjectCategories/*; do
    c=`basename $i`
    mkdir -p caltech_256_train_60/$c
    for j in `ls $i/*.jpg | shuf | head -n 60`; do
        mv $j caltech_256_train_60/$c/
    done
done

python ~/mxnet/tools/im2rec.py --list True --recursive True caltech-256-60-train caltech_256_train_60/
python ~/mxnet/tools/im2rec.py --list True --recursive True caltech-256-60-val 256_ObjectCategories/
python ~/mxnet/tools/im2rec.py --resize 256 --quality 90 --num-thread 16 caltech-256-60-val 256_ObjectCategories/
python ~/mxnet/tools/im2rec.py --resize 256 --quality 90 --num-thread 16 caltech-256-60-train caltech_256_train_60/
```

The following codes download the pre-generated rec files. It may take a few minutes.

In [1]:
import os, urllib
def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.urlretrieve(url, filename)
download('http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec')
download('http://data.mxnet.io/data/caltech-256/caltech-256-60-val.rec')

Next we define the function which returns the data iterators.

In [2]:
import mxnet as mx

def get_iterators(batch_size, data_shape=(3, 224, 224)):
    train = mx.io.ImageRecordIter(
        path_imgrec         = './caltech-256-60-train.rec',
        data_name           = 'data',
        label_name          = 'softmax_label',
        batch_size          = batch_size,
        data_shape          = data_shape,
        shuffle             = True,
        rand_crop           = True,
        rand_mirror         = True)
    val = mx.io.ImageRecordIter(
        path_imgrec         = './caltech-256-60-val.rec',
        data_name           = 'data',
        label_name          = 'softmax_label',
        batch_size          = batch_size,
        data_shape          = data_shape,
        rand_crop           = False,
        rand_mirror         = False)
    return (train, val)

Next we download a pretrained 50-layer ResNet model and load into memory 

In [3]:
def get_model(prefix, epoch):
    download(prefix+'-symbol.json')
    download(prefix+'-%04d.params' % (epoch,))

get_model('http://data.mxnet.io/models/imagenet/resnet/50-layers/resnet-50', 0)
sym, arg_params, aux_params = mx.model.load_checkpoint('resnet-50', 0)

## Train

We first define a function which replaces the the last fully-connected layer for a given network. 

In [4]:
def get_fine_tune_model(symbol, arg_params, num_classes, layer_name='flatten0'):
    """
    symbol: the pre-trained network symbol
    arg_params: the argument parameters of the pre-trained model
    num_classes: the number of classes for the fine-tune datasets
    layer_name: the layer name before the last fully-connected layer
    """
    all_layers = sym.get_internals()
    net = all_layers[layer_name+'_output']
    net = mx.symbol.FullyConnected(data=net, num_hidden=num_classes, name='fc1')
    net = mx.symbol.SoftmaxOutput(data=net, name='softmax')
    new_args = dict({k:arg_params[k] for k in arg_params if 'fc1' not in k})
    return (net, new_args)

Now we create a module. We first call `init_params` to randomly initialize parameters, next use `set_params` to replace all parameters except for the last fully-connected layer with pre-trained model. Then we can start training.

In [6]:
import logging
head = '%(asctime)-15s %(message)s'
logging.basicConfig(level=logging.DEBUG, format=head)

def fit(symbol, arg_params, aux_params, train, val, batch_size):
    mod = mx.mod.Module(symbol=new_sym, context=mx.gpu())
    mod.bind(data_shapes=train.provide_data, label_shapes=train.provide_label)
    mod.init_params(initializer=mx.init.Xavier(rnd_type='gaussian', factor_type="in", magnitude=2))
    mod.set_params(new_args, aux_params, allow_missing=True)
    mod.fit(train, val, 
        num_epoch=4,
        batch_end_callback = mx.callback.Speedometer(batch_size, 50),        
        optimizer='sgd',
        optimizer_params={'learning_rate':0.01},
        eval_metric='acc')    
    
batch_size = 32
(train, val) = get_iterators(batch_size)
(new_sym, new_args) = get_fine_tune_model(sym, arg_params, 256)
fit(new_sym, new_args, aux_params, train, val, batch_size)

2016-10-21 05:03:44,995 Already binded, ignoring bind()
2016-10-21 05:04:19,726 Epoch[0] Batch [50]	Speed: 47.31 samples/sec	Train-accuracy=0.017770
2016-10-21 05:04:53,733 Epoch[0] Batch [100]	Speed: 47.05 samples/sec	Train-accuracy=0.108750
2016-10-21 05:05:27,859 Epoch[0] Batch [150]	Speed: 46.89 samples/sec	Train-accuracy=0.218125
2016-10-21 05:06:01,975 Epoch[0] Batch [200]	Speed: 46.90 samples/sec	Train-accuracy=0.371875
2016-10-21 05:06:36,094 Epoch[0] Batch [250]	Speed: 46.90 samples/sec	Train-accuracy=0.423125
2016-10-21 05:07:10,221 Epoch[0] Batch [300]	Speed: 46.89 samples/sec	Train-accuracy=0.527500
2016-10-21 05:07:44,333 Epoch[0] Batch [350]	Speed: 46.91 samples/sec	Train-accuracy=0.563125
2016-10-21 05:08:18,438 Epoch[0] Batch [400]	Speed: 46.92 samples/sec	Train-accuracy=0.606250
2016-10-21 05:08:52,545 Epoch[0] Batch [450]	Speed: 46.91 samples/sec	Train-accuracy=0.651875
2016-10-21 05:09:13,694 Epoch[0] Train-accuracy=0.645161
2016-10-21 05:09:13,695 Epoch[0] Time cost


As can be seen, even for 4 data epochs, we can get 80% validation accuracy. It matches the state-of-the-art results training on caltech-256 alone, e.g. [78% by VGG group](http://www.robots.ox.ac.uk/~vgg/research/deep_eval/). 

Next we try to use another pretrained model, which use more 

In [ ]:
get_model('http://data.mxnet.io/models/imagenet-11k/resnet-152/resnet-152', 0)
sym, arg_params, aux_params = mx.model.load_checkpoint('resnet-152', 0)
(new_sym, new_args) = get_fine_tune_model(sym, arg_params, 256)
fit(new_sym, new_args, aux_params, train, val, batch_size)

2016-10-21 05:37:13,786 Already binded, ignoring bind()
2016-10-21 05:38:35,521 Epoch[0] Batch [50]	Speed: 20.12 samples/sec	Train-accuracy=0.322304
2016-10-21 05:39:55,822 Epoch[0] Batch [100]	Speed: 19.93 samples/sec	Train-accuracy=0.611875
2016-10-21 05:41:16,128 Epoch[0] Batch [150]	Speed: 19.92 samples/sec	Train-accuracy=0.678125
2016-10-21 05:42:36,450 Epoch[0] Batch [200]	Speed: 19.92 samples/sec	Train-accuracy=0.713750
2016-10-21 05:43:56,716 Epoch[0] Batch [250]	Speed: 19.93 samples/sec	Train-accuracy=0.706875
2016-10-21 05:45:17,020 Epoch[0] Batch [300]	Speed: 19.92 samples/sec	Train-accuracy=0.739375
2016-10-21 05:46:37,309 Epoch[0] Batch [350]	Speed: 19.93 samples/sec	Train-accuracy=0.743125
2016-10-21 05:47:57,625 Epoch[0] Batch [400]	Speed: 19.92 samples/sec	Train-accuracy=0.738750
2016-10-21 05:49:17,935 Epoch[0] Batch [450]	Speed: 19.92 samples/sec	Train-accuracy=0.746250
2016-10-21 05:50:07,737 Epoch[0] Train-accuracy=0.735887
2016-10-21 05:50:07,738 Epoch[0] Time cost